#Experiments 

<font size = 3px>This notebook was used to identify the architecture and model output sizes at each step of the architecture.
------



In [ ]:
import numpy as np
import torch
import csv
from torch import nn
import pandas as pd
import cv2 as cv
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, ZeroPad2d
from torchvision import transforms
import shutil
from sklearn.metrics import auc, confusion_matrix, classification_report
import seaborn as sns
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import random

In [ ]:
class smallModel(nn.Module):
  def __init__(self):
    super(smallModel, self).__init__()
    
    self.zp1 = nn.ZeroPad2d(1)
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
    self.mp1 = nn.MaxPool2d(kernel_size=2)
    self.dp = nn.Dropout(p=0.3)

    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
    self.mp2 = nn.MaxPool2d(kernel_size=2)
    #self.dp = nn.Dropout(p=0.3)

    self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
    self.mp3 = nn.MaxPool2d(kernel_size=2)
    #self.dp = nn.Dropout(p=0.3)

    self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
    self.mp4 = nn.MaxPool2d(kernel_size=2)

    self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
    self.mp5 = nn.MaxPool2d(kernel_size=2)

    self.conv = nn.Conv2d(128, 32, 1)

    self.relu = nn.ReLU()

    self.linear_layers = Sequential(
        Linear(128, 128)
    )

    """self.pad1 = nn.ZeroPad2d(1)
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
    self.mp1


    self.relu = nn.ReLU(inplace = True)"""
    self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

    self.encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=8)

  def forward(self, x):
    x = self.zp1(x)
    print("Padding: ", x.shape)
    x = self.mp1(self.relu(self.conv1(x)))
    x = self.dp(x)
    print("Layer 1: ", x.shape)

    #x = self.zp2(x)
    #print("Padding: ", x.shape)
    x = self.mp2(self.relu(self.conv2(x)))
    x = self.dp(x)
    print("Layer 2: ", x.shape)

    #x = self.zp3(x)
    #print("Padding: ", x.shape)
    x = self.mp3(self.relu(self.conv3(x)))
    x = self.dp(x)
    print("Layer 3: ", x.shape)

    x = self.mp4(self.relu(self.conv4(x)))
    x = self.dp(x)
    print("Layer 4: ", x.shape)

    x = self.mp5(self.relu(self.conv5(x)))
    x = self.dp(x)
    print("Layer 5: ", x.shape)

    """x = self.conv(x)
    print("Layer 6: ", x.shape)
    print(x)

    x = x.view(1, 1, 128)
    print("Layer 4: ", x.shape)
    print(x)

    x = self.encoder_layer(x)
    print("Layer 5: ", x.shape)"""
  
    return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = smallModel().to(device)
print(model)

smallModel(
  (zp1): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dp): Dropout(p=0.3, inplace=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (mp3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (mp4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (mp5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv): Conv2d(128, 32, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (linear_l

In [ ]:
x = torch.rand(8, 200, 200)
x = x.unsqueeze(1)
print(x)

In [ ]:
model.train()
pred = model(x)

Padding:  torch.Size([8, 1, 202, 202])
Layer 1:  torch.Size([8, 32, 101, 101])
Layer 2:  torch.Size([8, 64, 50, 50])
Layer 3:  torch.Size([8, 128, 25, 25])
Layer 4:  torch.Size([8, 256, 12, 12])
Layer 5:  torch.Size([8, 512, 6, 6])


In [ ]:
xtra = pred.flatten(2).permute(1, 0, )


In [ ]:
x = x.flatten(2).permute(2, 0, 1)#Before this operation, x.shape = [batch_size, 512, 6, 6]
                                 #After this operation, x.shape = [36, batch_size, 512] 

x = self.transformer_encoder(pos + x) #Broadcasting: [36, 1, 512] + [36, batch_size, 512]

x = x.permute(1, 2, 0)
x = torch.reshape(x, ((x.shape)[0], 512 * 36))

In [ ]:
print(pred.shape)
H = pred.shape[-1]#H = 6
W = pred.shape[-2]#H = 6
print(H)
print(W)

torch.Size([8, 512, 6, 6])
6
6


In [ ]:
col_embed = torch.rand(64, 512 // 2)
row_embed = torch.rand(64, 512 // 2)
#col_embed = col_embed[:W].unsqueeze(0).repeat(H, 1, 1) #.shape # .flatten(0, 1).unsqueeze(1)
print(col_embed.shape)
print(row_embed.shape)

torch.Size([64, 256])
torch.Size([64, 256])


In [ ]:
print(col_embed[:W].unsqueeze(0).repeat(H, 1, 1).shape)
print(row_embed[:H].unsqueeze(1).repeat(1, W, 1).shape)

torch.Size([6, 6, 256])
torch.Size([6, 6, 256])


In [ ]:
torch.cat([
    col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
    row_embed[:H].unsqueeze(1).repeat(1, W, 1),
], dim=-1).flatten(0, 1).unsqueeze(1).shape

torch.Size([36, 1, 512])

In [ ]:
src = pred.flatten(2).permute(2, 0, 1)
src.shape

torch.Size([36, 8, 512])

In [ ]:
pred.shape

torch.Size([8, 512, 6, 6])

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
#src = torch.rand(1, 1, 128)
xtra = encoder_layer(src)

In [ ]:
torch.reshape(x.permute(1, 2, 0), (8, 512 * 36)).shape

torch.Size([8, 18432])

In [ ]:
#H, W = x.shape[-2:]
pos = torch.cat([
    col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
    row_embed[:H].unsqueeze(1).repeat(1, W, 1),
], dim=-1).flatten(0, 1).unsqueeze(1)

In [ ]:
xtra.permute(1, 2, 0).shape
#x = torch.reshape(x, ((x.shape)[0], 512 * 36))

torch.Size([8, 512, 36])

In [ ]:
18432 -> 9216
9216 -> 4096
4096 -> 1024
1024 -> 62